# Importing necessary libraries


In [1]:
import os
import cv2
import torch
import json
from ultralytics import YOLO
import time

curr_path=os.getcwd()

# Loading all fake videos contaning folders



In [ ]:
fake_folders=[os.path.join(curr_path,file) for file in os.listdir(curr_path) if file.endswith("fake")]
fake_folders

# Loading all real videos contaning folders


In [2]:
real_folders=[os.path.join(curr_path,file) for file in os.listdir(curr_path) if file.endswith("real")]
real_folders

['c:\\kleos_dataset\\real\\celeb_df_real',
 'c:\\kleos_dataset\\real\\cropped_celeb_df_real',
 'c:\\kleos_dataset\\real\\dfdc_cropped_real',
 'c:\\kleos_dataset\\real\\dfdc_real']

# Loading all types of dataset(celeb df and dfdc) of fake videos


In [ ]:
celeb_df_fake_vids=[os.path.join(fake_folders[0],vid) for vid in os.listdir(fake_folders[0]) if vid.endswith(".mp4")]
dfdc_fake_vids=[os.path.join(fake_folders[1],vid) for vid in os.listdir(fake_folders[1]) if vid.endswith(".mp4")]
total_fake_vids=len(celeb_df_fake_vids)+len(dfdc_fake_vids)
total_fake_vids

# Loading all types of dataset(celeb df and dfdc) of real videos


In [ ]:
celeb_df_real_vids=[os.path.join(real_folders[0],vid) for vid in os.listdir(real_folders[0]) if vid.endswith(".mp4")]
celeb_df_real_vids
dfdc_real_vids=[os.path.join(real_folders[3],vid) for vid in os.listdir(real_folders[3]) if vid.endswith(".mp4")]
total_real_vids=len(celeb_df_real_vids)+len(dfdc_real_vids)
total_real_vids

# Frame extraction and saving in json file


In [ ]:
def extract_video_frames(video_path,output_dir):
    # print("for video",video_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    # print(total_frames,frame_rate)
    # print("duration",total_frames/frame_rate)

    #nor of continuous frames to group together)
    group_size = 8

    samples = []

    for i in range(0, total_frames - group_size + 1, group_size):
        frames = []
        for j in range(group_size):
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
            else:
                print(f"Error reading frame {i + j} from video {video_path}")
                break

        if len(frames) < group_size:  #if total frames not divisible by 8 ignore the ending frames

            break

        sample = {
            'video_path': video_path,
            'frame_range': (i + 1, i + group_size) 
        }
        samples.append(sample)
    
    # print(samples)

    cap.release()

    json_filename = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(video_path))[0]}.json")
    
    with open(json_filename, 'w') as json_file:
        json.dump(samples, json_file, indent=4)
    
    print(f"Saved {len(samples)} samples to {json_filename}")

output_dir=os.path.join((os.getcwd()),"celeb_df_real_json_files")
for vid in celeb_df_real_vids:
   extract_video_frames(vid,output_dir)

# Function to crop faces from videos using yolo-v8 followed by openCV built-in object tracker

In [ ]:

def run_tracker(filename, model,output_path):
    # print(filename)
    video = cv2.VideoCapture(filename)  
    tracker = None  
    detection_duration = 4  
    start_time = time.time()  

    # frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    # frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    req_resolution=(224,224)

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, req_resolution)

    while True:
        ret, frame = video.read() 
        if not ret:
            break
        
        if tracker is None:
            results = model(frame)
            if results and len(results) > 0:
                bboxes = results[0].boxes  

                if len(bboxes) > 0:
                    bbox = bboxes[0]
                    last_bbox = bbox  
                    x1, y1, x2, y2 = tuple(map(int, bbox.xyxy[0]))
                    w = x2 - x1 
                    h = y2 - y1 
                    
                    if time.time() - start_time >= detection_duration:
                        print(f"Initializing tracker with bbox: (x1={x1}, y1={y1}, w={w}, h={h})")
                        tracker = cv2.TrackerKCF_create()
                        success = tracker.init(frame, (x1, y1, w, h))

                        if not success:
                            print("Tracker not initialized")
                        else:
                            print("Tracker initialized successfully.")
                    else:
                        print("Detecting with YOLO...")
                    
                    x1, y1, x2, y2 = tuple(map(int, last_bbox.xyxy[0]))
                    w = x2 - x1 
                    h = y2 - y1                     
                    face = frame[y1:y1+h, x1:x1+w]
                    face_resized = cv2.resize(face, req_resolution) #,interpolation=cv2.INTER_AREA)
                    out.write(face_resized)

        else:
            print("Tracking in progress...")
            success, bbox = tracker.update(frame)
            
            if success:
                (x, y, w, h) = tuple(map(int, bbox))
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                face = frame[y:y+h, x:x+w]

                face_resized = cv2.resize(face, req_resolution) #,interpolation=cv2.INTER_AREA)
                out.write(face_resized)
            else:
                print("Tracker update failed.")
                tracker = None 
        cv2.imshow(f"Tracking_Stream", frame)

        key = cv2.waitKey(1)
        if key == ord("q"):
            break
    print("done cropping video",os.path.basename(filename))
    video.release()
    out.release()
    cv2.destroyAllWindows()




# cv2.destroyAllWindows()


# Cropping all videos

In [24]:
model = YOLO("yolov8n-face.pt")


for vid in celeb_df_real_vids:
  op_vid=f"cropped_celeb_df_real//{os.path.basename(vid)}"
  run_tracker(vid,model,op_vid)

for vid in celeb_df_fake_vids:
  op_vid=f"cropped_celeb_df_fake//{os.path.basename(vid)}"
  run_tracker(vid,model,op_vid)

for vid in dfdc_fake_vids:
  op_vid=f"dfdc_cropped_fake//{os.path.basename(vid)}"
  run_tracker(vid,model,op_vid)

for vid in dfdc_real_vids:
  op_vid=f"dfdc_cropped_real//{os.path.basename(vid)}"
  run_tracker(vid,model,op_vid)



'id0_id16_0003.mp4'